In [1]:
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import time

import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score

import random, os, json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, GRU, Dropout, Dense
from tensorflow.keras import backend as K


from joblib import Parallel, delayed
import multiprocessing

import pickle
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

### FUNCTIONS OF THE MODEL

In [ ]:
def reset_keras(seed=42):
    """Function to ensure that results from Keras models
    are consistent and reproducible across different runs"""
    
    K.clear_session()
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    random.seed(seed)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed)
    # 4. Set `tensorflow` pseudo-random generator at a fixed value
    tf.random.set_seed(seed)
    
def build_model(hyperparameters):
    """
    Builds a GRU model based on several hyperparameters.

    Args:
        - hyperparameters: Dictionary containing the model hyperparameters. 
    Returns:
        - model: A tf.keras.Model with the compiled model.
    """
    hyperparameters['layers'] = [80, hyperparameters['middle_layer_dim'], 1]
    l2_lambda = hyperparameters.get("l2_lambda", 1e-4)
    
    dynamic_input = tf.keras.layers.Input(shape=(hyperparameters["n_time_steps"], hyperparameters["layers"][0]))
    masked = tf.keras.layers.Masking(mask_value=hyperparameters['mask_value'])(dynamic_input)
    optimizer = Adam(learning_rate=hyperparameters["lr_scheduler"], weight_decay=hyperparameters["weight_decay"])

    gru_encoder = tf.keras.layers.LSTM(
        hyperparameters["layers"][1],
        dropout=hyperparameters['dropout'],
        return_sequences=False,
        activation=hyperparameters['activation'],
        kernel_regularizer=regularizers.l2(l2_lambda),
        use_bias=False
    )(masked)

    if hyperparameters['dropout'] > 0.0:
        gru_encoder = tf.keras.layers.Dropout(hyperparameters['dropout'])(gru_encoder)

    output = tf.keras.layers.Dense(1, use_bias=False, activation="sigmoid",kernel_regularizer=regularizers.l2(l2_lambda))(gru_encoder)

    model = tf.keras.Model(dynamic_input, output)
    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy', "AUC"], weighted_metrics = []
    )
        
    return model



In [ ]:
def run_network(X_train, X_val, y_train, y_val, 
                hyperparameters, seed):
    """
    Trains and evaluates the built GRU model based on the provided data and hyperparameters.

    Args:
        - X_train, X_val, y_train, y_val: numpy.ndarray. Training (T) and Validation (V) data labels.
        - hyperparameters: Dictionary containing training and model hyperparameters.
        - seed: Random seed for reproducibility.
    Returns:
        - model (tf.keras.Model): The trained Keras model.
        - hist (tf.keras.callbacks.History): Training history object containing loss and metrics.
        - training_time (float): Total training time in seconds.
    """

    model = None
    model = build_model(hyperparameters)
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  min_delta=hyperparameters["mindelta"],
                                                  patience=hyperparameters["patience"],
                                                  restore_best_weights=True,
                                                  mode="min")
    start_time = time.time()
    
    hist = model.fit(X_train, y_train,
                     validation_data=(X_val, y_val),
                     callbacks=[earlystopping], batch_size=hyperparameters['batch_size'], epochs=hyperparameters['n_epochs_max'],
                     verbose=hyperparameters['verbose'])
    
    end_time = time.time()
    training_time = end_time - start_time

    return model, hist, training_time


In [ ]:
def objective(trial, hyperparameters, seed, X_train, y_train, X_val, y_val, split, norm, n_time_steps):
    """
    Objective function for hyperparameter optimization using Optuna.
    Args:
        - trial (optuna.trial.Trial): Optuna trial object.
        - X_train, X_val, y_train, y_val: numpy.ndarray. Training (T) and Validation (V) data labels.
        - hyperparameters: Dictionary containing training and model hyperparameters.
        - seed: Random seed for reproducibility.  
        - split: String indicating the data split.
        - norm: String with the type of normalization applied to the data.
        - n_time_steps: Number of time steps in the input.    
    Returns:
        - metric_dev (float): Best validation loss   
    """

    print(f"Trial {trial.number} started")
    hyperparameters_copy = hyperparameters.copy()

    hyperparameters_copy["dropout"] = trial.suggest_float('dropout', 0.0, 0.3)
    middle_dim = trial.suggest_int('middle_layer_dim', 2, 50, step=2)
    hyperparameters_copy['middle_layer_dim'] = middle_dim
    hyperparameters_copy["lr_scheduler"] = trial.suggest_loguniform('lr_scheduler', 1e-3, 1e-1)
    hyperparameters_copy["l2_lambda"] = trial.suggest_loguniform('l2_lambda', 1e-6, 1e-2)
    hyperparameters_copy["adjustment_factor"] = trial.suggest_categorical('adjustment_factor', [1])
    hyperparameters_copy["activation"] = trial.suggest_categorical('activation', ['LeakyReLU', 'tanh'])
    hyperparameters_copy['patience'] = trial.suggest_int('patience', 1, 50)
    hyperparameters_copy['mindelta'] = trial.suggest_loguniform('mindelta', 1e-10, 1e-3)
    hyperparameters_copy["weight_decay"] = trial.suggest_loguniform('weight_decay', 1e-5, 0)
    

    hyperparameters_copy['batch_size'] = hyperparameters['batch_size']
    hyperparameters_copy['n_epochs_max'] = hyperparameters['n_epochs_max']
   
    v_val_loss = []
        

    model, hist, training_time = run_network(
            X_train, X_val,
            y_train,
            y_val,
            hyperparameters_copy,
            seed
    )

    v_val_loss.append(np.min(hist.history["val_loss"]))

    metric_dev = np.mean(v_val_loss)
    return metric_dev

In [ ]:
def optuna_study(hyperparameters, seed, X_train, y_train, X_val, y_val, split, norm, n_time_steps):
    """
    Runs an Optuna study to optimize hyperparameters for the model.
    
    Args:
        - X_train, X_val, y_train, y_val: numpy.ndarray. Training (T) and Validation (V) data labels.
        - hyperparameters: Dictionary containing training and model hyperparameters.
        - seed: Random seed for reproducibility.  
        - split: String indicating the data split.
        - norm: String with the type of normalization applied to the data.
        - n_time_steps: Number of time steps in the input.       
    Returns:
        - best_hyperparameters: Dictionary containing the best hyperparameters found 
          after the optimization process.
    """
    sampler = optuna.samplers.TPESampler(seed=42)
    study = optuna.create_study(direction='minimize', sampler=sampler)
    
    study.optimize(lambda trial: objective(trial, hyperparameters, seed, X_train, y_train , X_val, y_val, split, norm, n_time_steps), n_trials=30)
    
    best_params = study.best_params
    best_metric = study.best_value
    
    layers = [80, best_params['middle_layer_dim'], 1]
    
    best_hyperparameters = {
        'dropout': best_params['dropout'],
        'middle_layer_dim': best_params['middle_layer_dim'],
        'layers': layers,
        'lr_scheduler': best_params['lr_scheduler'],
        'l2_lambda': best_params['l2_lambda'],
        'adjustment_factor': best_params['adjustment_factor'],
        'activation': best_params['activation'],
        'batch_size': hyperparameters['batch_size'],
        'n_epochs_max': hyperparameters['n_epochs_max'],
        'patience': best_params['patience'],
        'mindelta': best_params['mindelta'],
        'weight_decay': best_params['weight_decay']
    }
    

    print(f"Best Hyperparameters: {best_params}")
    print(f"Best Validation Metric: {best_metric}")

    return best_hyperparameters


### HYPERPARAMETERS 

- **seeds**: Seed values to ensure reproducibility.
- **input_shape**: Number of features in each time step of the input data.
- **n_time_steps**: Number of time steps in the input sequence.
- **batch_size**: Number of batches for training.
- **norm**: Type of normalization applied to the data.
- **dropout**: Dropout rate to prevent overfitting.
- **l2_lambda**: L2 regularization coefficient.
- **lr_scheduler**: Learning rate assigned to the optimizer.
- **patience**: Number of epochs with no improvement before early stopping is triggered.
- **weight_decay**: Weight decay for the optimizer to apply additional L2 regularization on weights.
- **middle_layer_dim**: Different configurations for the middle layer of the model.
- **mindelta**: Minimum delta required to consider as an improvement.

In [ ]:
seeds = [42, 76, 124, 163, 192, 205]

input_shape = 80
n_time_steps = 14
batch_size = 32
n_epochs_max = 1000

adjustment_factor = [1]  
activation = ['LeakyReLU']
norm = "robustNorm"
patience = 3  
monitor = "val_loss"   

hyperparameters = {
    "n_time_steps": n_time_steps,
    "mask_value": 666,
    "batch_size": batch_size,
    "n_epochs_max": n_epochs_max,
    "monitor": monitor,
    "mindelta": 0,
    "patience": patience,
    "dropout": 0.2,
    "l2_lambda": 1e-4,
    "verbose": 1
}

### PREDICTIONS

In [ ]:
run_model = True
debug = True
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
)

if run_model:
    import time

    loss_train = []
    loss_dev = []
    v_models = []
    training_times = []
    optimization_times = []
    inference_times = []

    v_accuracy_test = []
    v_specificity = []
    v_precision = []
    v_recall = []
    v_f1score = []
    v_roc = []
    v_aucpr = []

    bestHyperparameters_bySplit = {}
    y_pred_by_split = {}
    results = ""

    for i in [1, 2, 3]:
        print("====================>", i)


        X_train = np.load(f"../../DATA/s{i}/X_train_tensor_robustNorm.npy")
        X_val = np.load(f"../../DATA/s{i}/X_val_tensor_robustNorm.npy")

        y_train = pd.read_csv(f"../../DATA/s{i}/y_train_robustNorm.csv")[['individualMRGerm_stac']]
        y_train = y_train.iloc[0:y_train.shape[0]:hyperparameters["n_time_steps"]].reset_index(drop=True)

        y_val = pd.read_csv(f"../../DATA/s{i}/y_val_robustNorm.csv")[['individualMRGerm_stac']]
        y_val = y_val.iloc[0:y_val.shape[0]:hyperparameters["n_time_steps"]].reset_index(drop=True)

        X_test = np.load(f"../../DATA/s{i}/X_test_tensor_robustNorm.npy")
        y_test = pd.read_csv(f"../../DATA/s{i}/y_test_robustNorm.csv")[['individualMRGerm_stac']]
        y_test = y_test.iloc[0:y_test.shape[0]:hyperparameters["n_time_steps"]].reset_index(drop=True)


        start_opt = time.time()
        bestHyperparameters = optuna_study(
            hyperparameters,
            seeds[i-1],
            X_train, y_train,  
            X_val, y_val,
            f"s{i}",
            norm,
            n_time_steps
        )
        end_opt = time.time()
        optimization_times.append(end_opt - start_opt)

        print(f"Best layers: {bestHyperparameters['layers']}")
        bestHyperparameters_bySplit[str(i)] = bestHyperparameters


        split_directory = f'./Results_LSTM_optuna/split_{i}'
        os.makedirs(split_directory, exist_ok=True)
        with open(os.path.join(split_directory, f"bestHyperparameters_split_{i}.pkl"), 'wb') as f:
            pickle.dump(bestHyperparameters, f)

        hyperparameters.update({
            "dropout": bestHyperparameters["dropout"],
            "layers": bestHyperparameters["layers"],
            "lr_scheduler": bestHyperparameters["lr_scheduler"],
            "l2_lambda": bestHyperparameters["l2_lambda"],
            "adjustment_factor": bestHyperparameters["adjustment_factor"],
            "activation": bestHyperparameters["activation"], 
            "patience": bestHyperparameters["patience"], 
            "weight_decay": bestHyperparameters["weight_decay"],
            "mindelta": bestHyperparameters["mindelta"],
            "middle_layer_dim": bestHyperparameters["middle_layer_dim"]
        })


        reset_keras()
        print(hyperparameters)


        start_train = time.time()
        model, hist, training_time = run_network(
            X_train, X_val,
            y_train, 
            y_val,
            hyperparameters,
            seeds[i-1]
        )
        end_train = time.time()
        training_times.append(end_train - start_train)

        v_models.append(model)
        loss_train.append(hist.history['loss'])
        loss_dev.append(hist.history['val_loss'])


        start_inf = time.time()
        y_pred = model.predict(x=X_test)
        end_inf = time.time()
        inference_times.append(end_inf - start_inf)

        y_pred_by_split[str(i)] = y_pred
        with open(os.path.join(split_directory, f"y_pred_split_{i}.pkl"), 'wb') as f:
            pickle.dump(y_pred, f)

        model.save(os.path.join(split_directory, f"model_split_{i}.h5"))


        accuracy_test = accuracy_score(y_test, np.round(y_pred))
        tn, fp, fn, tp = confusion_matrix(y_test, np.round(y_pred)).ravel()
        roc = roc_auc_score(y_test, y_pred)
        aucpr = average_precision_score(y_test, y_pred)

        v_accuracy_test.append(accuracy_test)
        v_specificity.append(tn / (tn + fp))
        v_precision.append(tp / (tp + fp))
        v_recall.append(tp / (tp + fn))
        v_f1score.append((2 * v_recall[i - 1] * v_precision[i - 1]) / (v_recall[i - 1] + v_precision[i - 1]))
        v_roc.append(roc)
        v_aucpr.append(aucpr)

        if debug:
            results += f"\tSplit {i} - Timing (s):\n"
            results += f"\t\tOptimization: {optimization_times[-1]:.2f}\n"
            results += f"\t\tTraining: {training_times[-1]:.2f}\n"
            results += f"\t\tInference: {inference_times[-1]:.2f}\n"
            results += f"\t\tTP: {tp} | FP: {fp} | TN: {tn} | FN: {fn}\n"
            results += f"\t\tAccuracy: {accuracy_test:.4f} | ROC-AUC: {roc:.4f} | AUC-PR: {aucpr:.4f}\n"


    directory = './Results_LSTM_optuna'
    os.makedirs(directory, exist_ok=True)
    summary_df = pd.DataFrame({
        "Split": [i for i in range(1, len(v_accuracy_test) + 1)],
        "OptimizationTime": optimization_times,
        "TrainingTime": training_times,
        "InferenceTime": inference_times,
        "Accuracy": v_accuracy_test,
        "Specificity": v_specificity,
        "Precision": v_precision,
        "Recall": v_recall,
        "F1Score": v_f1score,
        "ROC_AUC": v_roc,
        "AUC_PR": v_aucpr
    })

    summary_path = os.path.join(directory, "summary_metrics.csv")
    summary_df.to_csv(summary_path, index=False)


## RESULTS (PERFORMANCE)

In [2]:
directory = './Results_LSTM_optuna'
summary_path = os.path.join(directory, "summary_metrics.csv")
summary_df = pd.read_csv(summary_path)


def calculateKPI(parameter):
    """
    This function calculate the mean and deviation of a set of values of
    a given performance indicator.
    
    Returns: Mean and std (float)
    """
    mean = round(np.mean(parameter)*100, 2)
    deviation = round(np.sqrt(np.sum(np.power(parameter - np.mean(parameter), 2) / len(parameter)))*100, 2)
    return mean, deviation

def format_metric_line(metric_name, mean_value, deviation_value):
    return f"{metric_name}: {mean_value:.2f} +- {deviation_value:.2f}\n"

mean_test, deviation_test = calculateKPI(summary_df["Accuracy"])
mean_specificity, deviation_specificity = calculateKPI(summary_df["Specificity"])
mean_recall, deviation_recall = calculateKPI(summary_df["Recall"])
mean_f1, deviation_f1 = calculateKPI(summary_df["F1Score"])
mean_precision, deviation_precision = calculateKPI(summary_df["Precision"])
mean_roc, deviation_roc = calculateKPI(summary_df["ROC_AUC"])
mean_aucpr, deviation_aucpr = calculateKPI(summary_df["AUC_PR"])  

results = ""
results += format_metric_line("Test Accuracy", mean_test, deviation_test)
results += format_metric_line("Specificity", mean_specificity, deviation_specificity)
results += format_metric_line("Sensitivity", mean_recall, deviation_recall)
results += format_metric_line("Precision", mean_precision, deviation_precision)
results += format_metric_line("F1-score", mean_f1, deviation_f1)
results += format_metric_line("ROC-AUC", mean_roc, deviation_roc)
results += format_metric_line("AUC-PR", mean_aucpr, deviation_aucpr) 

final_results = (
    f"Sensitivity: {mean_recall:.2f} +- {deviation_recall:.2f}\n"
    f"Specificity: {mean_specificity:.2f} +- {deviation_specificity:.2f}\n"
    f"Precision: {mean_precision:.2f} +- {deviation_precision:.2f}\n"
    f"F1-score: {mean_f1:.2f} +- {deviation_f1:.2f}\n"
    f"ROC-AUC: {mean_roc:.2f} +- {deviation_roc:.2f}\n"
    f"AUC-PR: {mean_aucpr:.2f} +- {deviation_aucpr:.2f}\n" 
    f"Test Accuracy: {mean_test:.2f} +- {deviation_test:.2f}\n"
)

print(final_results)

Sensitivity: 67.92 +- 2.67
Specificity: 80.36 +- 2.22
Precision: 38.31 +- 1.95
F1-score: 48.92 +- 1.33
ROC-AUC: 75.68 +- 1.23
AUC-PR: 46.49 +- 3.22
Test Accuracy: 78.48 +- 1.59

